In [1]:
import configparser
import json
import os
import random
import time
import requests
import pandas as pd
import confuse
from pathlib import Path

In [30]:
# config = confuse.Configuration('master_thesis')
# config.set_file('../../config/config.yaml')
# print(config.config_dir())

In [5]:
# def get_historical_data(symbol, market, api_key):
#     api_url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={symbol}&market={market}&apikey={api_key}'
#     raw = requests.get(api_url).json()
#     return raw 

In [6]:
# list_of_coins = pd.read_csv('../data/digital_currency_list.csv')['currency code'].to_list()
# my_list_of_coins = random.sample(list_of_coins, 10)
# my_list_of_coins

In [7]:
# %%time
# for coin in my_list_of_coins:
#     time.sleep(20) #free account API restrictions
#     tmp = get_historical_data(coin, 'PLN', API_KEY)
#     path_to_save = '../data/historical'
#     with open('{}/{}_historical_prices.json'.format(path_to_save, coin), 'w') as f:
#         json.dump(tmp, f)
#     print('{} is uploaded'.format(coin))
#     print(tmp['Meta Data'])
   

In [9]:
BASE_URL = 'https://api.coingecko.com/api/v3/'

def simple_request(req):
    api_url = BASE_URL + req
    raw = requests.get(api_url).json()
    return raw 

In [11]:

names_of_coins = [item['id'] for item in  simple_request('coins/list')]

In [15]:
url = 'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=CNY&apikey=1C2AEN3RKKKFEUVN'
r = requests.get(url)
data = r.json()

print(data)

{'Meta Data': {'1. Information': 'Daily Prices and Volumes for Digital Currency', '2. Digital Currency Code': 'BTC', '3. Digital Currency Name': 'Bitcoin', '4. Market Code': 'CNY', '5. Market Name': 'Chinese Yuan', '6. Last Refreshed': '2021-11-28 00:00:00', '7. Time Zone': 'UTC'}, 'Time Series (Digital Currency Daily)': {'2021-11-28': {'1a. open (CNY)': '349835.22259200', '1b. open (USD)': '54716.47000000', '2a. high (CNY)': '350503.54560000', '2b. high (USD)': '54821.00000000', '3a. low (CNY)': '346972.80787200', '3b. low (USD)': '54268.77000000', '4a. close (CNY)': '349632.92908800', '4b. close (USD)': '54684.83000000', '5. volume': '1025.29206000', '6. market cap (USD)': '1025.29206000'}, '2021-11-27': {'1a. open (CNY)': '343487.97619200', '1b. open (USD)': '53723.72000000', '2a. high (CNY)': '353438.20800000', '2b. high (USD)': '55280.00000000', '3a. low (CNY)': '342760.89600000', '3b. low (USD)': '53610.00000000', '4a. close (CNY)': '349864.37740800', '4b. close (USD)': '54721.03

In [16]:
data

{'Meta Data': {'1. Information': 'Daily Prices and Volumes for Digital Currency',
  '2. Digital Currency Code': 'BTC',
  '3. Digital Currency Name': 'Bitcoin',
  '4. Market Code': 'CNY',
  '5. Market Name': 'Chinese Yuan',
  '6. Last Refreshed': '2021-11-28 00:00:00',
  '7. Time Zone': 'UTC'},
 'Time Series (Digital Currency Daily)': {'2021-11-28': {'1a. open (CNY)': '349835.22259200',
   '1b. open (USD)': '54716.47000000',
   '2a. high (CNY)': '350503.54560000',
   '2b. high (USD)': '54821.00000000',
   '3a. low (CNY)': '346972.80787200',
   '3b. low (USD)': '54268.77000000',
   '4a. close (CNY)': '349632.92908800',
   '4b. close (USD)': '54684.83000000',
   '5. volume': '1025.29206000',
   '6. market cap (USD)': '1025.29206000'},
  '2021-11-27': {'1a. open (CNY)': '343487.97619200',
   '1b. open (USD)': '53723.72000000',
   '2a. high (CNY)': '353438.20800000',
   '2b. high (USD)': '55280.00000000',
   '3a. low (CNY)': '342760.89600000',
   '3b. low (USD)': '53610.00000000',
   '4a. c

In [10]:
# list_of_coins = simple_request('simple/supported_vs_currencies')
# list_of_coins

In [11]:
btc = simple_request('coins/bitcoin/market_chart?vs_currency=USD&days=max&interval=daily')
btc['prices']

[[1367107200000, 135.3],
 [1367193600000, 141.96],
 [1367280000000, 135.3],
 [1367366400000, 117.0],
 [1367452800000, 103.43],
 [1367539200000, 91.01],
 [1367625600000, 111.25],
 [1367712000000, 116.79],
 [1367798400000, 118.33],
 [1367884800000, 106.4],
 [1367971200000, 112.64],
 [1368057600000, 113.0],
 [1368144000000, 118.78],
 [1368230400000, 113.01],
 [1368316800000, 114.713],
 [1368403200000, 117.18],
 [1368489600000, 114.5],
 [1368576000000, 114.156],
 [1368662400000, 115.5],
 [1368748800000, 123.1],
 [1368835200000, 123.88],
 [1368921600000, 120.501],
 [1369008000000, 122.58],
 [1369094400000, 122.9],
 [1369180800000, 123.0],
 [1369267200000, 125.748],
 [1369353600000, 131.7],
 [1369440000000, 130.77],
 [1369526400000, 134.6],
 [1369612800000, 128.985],
 [1369699200000, 129.179],
 [1369785600000, 132.13],
 [1369872000000, 127.401],
 [1369958400000, 127.98],
 [1370044800000, 129.09],
 [1370131200000, 121.3],
 [1370217600000, 121.16],
 [1370390400000, 121.309],
 [1370476800000, 1

In [30]:
btc = simple_request('coins/bitcoin/history?date=31-12-2020')

In [31]:
full_list = simple_request('/coins/list')

In [32]:
names_of_coins = [item['id'] for item in  full_list]
sampled_list = random.sample(names_of_coins, 25)

In [33]:
sampled_list

['aave-amm-uniwbtcweth',
 'substratum',
 'babelfish',
 'electric-arena',
 'zent-cash',
 'iron-bank-euro',
 'cardwallet',
 'elya',
 'genes-chain',
 'loanburst',
 'aventus',
 'hyper-finance',
 'basis-cash',
 'sovi-token',
 'complifi',
 'eunomia',
 'exclusivecoin',
 'odinycoin',
 'jewel',
 'handshake',
 'bishu-finance',
 'mobius-finance',
 'only1',
 'helium',
 'facts']

In [18]:
def get_historical_data(coin: str, date_start: str, date_finish: str):
    hist_date = {}
    dates = pd.date_range(date_start, date_finish).strftime('%d-%m-%Y').to_list()
    for date in dates:
        req = 'https://api.coingecko.com/api/v3/coins/{}/history?date={}'.format(coin,date)
        try:
            res = requests.get(req).json()
            if 'market_data' in res.keys():
                hist_date[date] = res
            else:
                pass
        except:
            time.sleep(10)
            pass
            
    return hist_date

In [34]:
get_historical_data('mars-panda-world', '10-11-2021', '10-15-2021')

{'11-10-2021': {'id': 'mars-panda-world',
  'symbol': 'mpt',
  'name': 'Mars Panda World',
  'localization': {'en': 'Mars Panda World',
   'de': 'Mars Panda World',
   'es': 'Mars Panda World',
   'fr': 'Mars Panda World',
   'it': 'Mars Panda World',
   'pl': 'Mars Panda World',
   'ro': 'Mars Panda World',
   'hu': 'Mars Panda World',
   'nl': 'Mars Panda World',
   'pt': 'Mars Panda World',
   'sv': 'Mars Panda World',
   'vi': 'Mars Panda World',
   'tr': 'Mars Panda World',
   'ru': 'Mars Panda World',
   'ja': 'Mars Panda World',
   'zh': 'Mars Panda World',
   'zh-tw': 'Mars Panda World',
   'ko': 'Mars Panda World',
   'ar': 'Mars Panda World',
   'th': 'Mars Panda World',
   'id': 'Mars Panda World'},
  'image': {'thumb': 'https://assets.coingecko.com/coins/images/18608/thumb/1CAXRy7m_400x400.jpg?1632667629',
   'small': 'https://assets.coingecko.com/coins/images/18608/small/1CAXRy7m_400x400.jpg?1632667629'},
  'market_data': {'current_price': {'aed': 0.8975595990058327,
    '